In [1]:
import sys
sys.path.insert(0, '../caffe/python')
sys.path.insert(0, '../lib/data_layer/')
sys.path.insert(0, '../lib/data/')
sys.path.insert(0, '../lib/util/')
import caffe

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
prototxt = '../models/hand_lstm/hand_lstm.prototxt'
solver_prototxt = '../models/hand_lstm/solver_hand_lstm.prototxt'
weight_ = '../weights/hand_lstm/hand_lstm_iter_160000.caffemodel'
net = caffe.Net(prototxt, weight_, caffe.TEST)

size of dataset is 2440 and 5812
phase: test
Outputs:  ['depth', 'joint', 'clip_markers', 'com', 'config', 'inds']


In [3]:
import time
t_start = time.time()

predicted_joints = np.array([None]* 8252)

for i in xrange(np.int(np.ceil(8252./ net.blobs['inds'].data.shape[0]))):
    net.forward()
    print 'iter = ', i
    for j, ind in enumerate(net.blobs['inds'].data):
        row = j / 32
        col = j % 32
        #print 'j = ', ind
        #print 'ind = ',ind
        #print 'gt joint = ', net.blobs['reshape-joint'].data[row][col].reshape(14, 3)
        #print 'pred joint = ', net.blobs['pred_joint'].data[row][col].reshape(14, 3)
        #gt_joint = (net.blobs['reshape-joint'].data[row][col].reshape(14, 3) * net.blobs['config'].data[0][0]/2 + net.blobs['com'].data[0].reshape(1, 3)).copy()
        if predicted_joints[int(ind) - 1] == None:
            predicted_joints[int(ind) - 1] = (net.blobs['pred_joint'].data[row][col].reshape(14, 3)  * net.blobs['config'].data[0][0]/2 + net.blobs['com'].data[0].reshape(1, 3)).copy()
        #print 'gt3D joint = ', gt_joint
        #print 'pred joint = ', predicted_joints[int(ind) - 1]
        #input()
t_end = time.time()
print 'time elapse {}'.format((t_end - t_start) / 8252)

iter =  0
iter =  1
iter =  2
iter =  3
iter =  4
iter =  5
iter =  6
iter =  7
iter =  8
iter =  9
iter =  10
iter =  11
iter =  12
iter =  13
iter =  14
iter =  15
iter =  16
time elapse 0.110225968984


/home/wuyiming/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:17: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.


In [77]:
net.blobs['config'].data

array([[ 249.,  249.,  249.],
       [ 249.,  249.,  249.],
       [ 249.,  249.,  249.],
       ..., 
       [ 300.,  300.,  300.],
       [ 300.,  300.,  300.],
       [ 300.,  300.,  300.]], dtype=float32)

In [76]:
np.sum(np.sqrt(((net.blobs['pred_joint'].data - net.blobs['reshape-joint'].data)*net.blobs['config'].data[-1][0])**2))/32

0.32711517810821533

In [86]:
np.sqrt((((net.blobs['pred_joint'].data - net.blobs['reshape-joint'].data)*net.blobs['config'].data[-1][0])**2).sum(2))/42

array([[ 1.77600718,  1.29080176,  1.77747178,  1.57364583,  1.25077868,
         1.42750573,  1.3564688 ,  1.1798017 ,  1.61842978,  1.71955383,
         2.09182119,  1.27257216,  1.06166816,  1.44716096,  1.86835611,
         2.10498691,  1.65279365,  1.96046793,  1.50495148,  1.70698643,
         1.18960261,  1.68627262,  1.33763468,  1.5957073 ,  1.61755276,
         1.54820192,  1.25627863,  1.50453675,  1.85500848,  1.79321194,
         1.57422113,  1.50421274],
       [ 0.92352313,  1.09959579,  1.17120826,  1.41487718,  0.84109408,
         0.71042186,  0.79961556,  0.75366104,  0.83480382,  0.91303623,
         1.20748615,  0.94876087,  0.83406466,  0.86606789,  1.10188043,
         1.5781337 ,  1.16283631,  1.04908943,  0.79920089,  1.04119718,
         0.63217038,  0.90378761,  0.86806381,  0.99001503,  1.32267129,
         1.02185333,  0.78192747,  0.93497467,  1.1803081 ,  1.6392324 ,
         1.35938823,  0.98201019],
       [ 1.1424284 ,  0.97993839,  0.86006546,  1.1769

In [74]:
net.blobs['reshape-joint'].data.shape

(16, 32, 42)

In [82]:
a = np.array([[1,2,3,4],[2,3,4,5]])
np.sum(a, axis=)

SyntaxError: non-keyword arg after keyword arg (<ipython-input-82-b620ce646fdc>, line 2)

In [7]:
with open('../result/OURS/hand_lstm_160000.txt', 'w') as f:
    for i in xrange(predicted_joints.shape[0]):
        for item in predicted_joints[i].reshape(14*3):
            f.write("%s "% item)
        f.write("\n")

In [ ]:
solver = None
solver = caffe.AdamSolver(solver_prototxt)
solver.net.forward()
solver.test_nets[0].forward()

In [25]:
def vis_square(data):
    """Take an array of shape (n, height, width) or (n, height, width, 3)
       and visualize each (height, width) thing in a grid of size approx. sqrt(n) by sqrt(n)"""
    
    # normalize data for display
    data = (data - data.min()) / (data.max() - data.min())
    
    # force the number of filters to be square
    n = int(np.ceil(np.sqrt(data.shape[0])))
    padding = (((0, n ** 2 - data.shape[0]),
               (0, 1), (0, 1))                 # add some space between filters
               + ((0, 0),) * (data.ndim - 3))  # don't pad the last dimension (if there is one)
    data = np.pad(data, padding, mode='constant', constant_values=1)  # pad with ones (white)

    # tile the filters into an image
    data = data.reshape((n, n) + data.shape[1:]).transpose((0, 2, 1, 3) + tuple(range(4, data.ndim + 1)))
    data = data.reshape((n * data.shape[1], n * data.shape[3]) + data.shape[4:])
    
    plt.imshow(data); plt.axis('off')

In [69]:
net.blobs['fc1'].data[1]

array([-0.01275859, -0.02090291,  0.12513016, ...,  0.00697055,
        0.02041021,  0.08731566], dtype=float32)

In [68]:
net.blobs['reshape-data'].data[0][1]

array([-0.01275859, -0.02090291,  0.12513016, ...,  0.00697055,
        0.02041021,  0.08731566], dtype=float32)

In [ ]:
plt.hist(net.blobs['lstm'].data[0])

In [43]:
help(plt.bar)

Help on function bar in module matplotlib.pyplot:

bar(left, height, width=0.8, bottom=None, hold=None, data=None, **kwargs)
    Make a bar plot.
    
    Make a bar plot with rectangles bounded by:
    
      `left`, `left` + `width`, `bottom`, `bottom` + `height`
            (left, right, bottom and top edges)
    
    Parameters
    ----------
    left : sequence of scalars
        the x coordinates of the left sides of the bars
    
    height : sequence of scalars
        the heights of the bars
    
    width : scalar or array-like, optional
        the width(s) of the bars
        default: 0.8
    
    bottom : scalar or array-like, optional
        the y coordinate(s) of the bars
        default: None
    
    color : scalar or array-like, optional
        the colors of the bar faces
    
    edgecolor : scalar or array-like, optional
        the colors of the bar edges
    
    linewidth : scalar or array-like, optional
        width of bar edge(s). If None, use default
      